In [1]:
import pandas as pd
from pathlib import Path


In [2]:
data_path = Path("bmw.csv")
df = pd.read_csv(data_path)
df.head()


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


In [3]:
# Initial Data Inspection (to understand: num of rows and columns, data types and unexpexted values)
print("rows, cols:", df.shape)
df.info()


rows, cols: (10781, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10781 entries, 0 to 10780
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10781 non-null  object 
 1   year          10781 non-null  int64  
 2   price         10781 non-null  int64  
 3   transmission  10781 non-null  object 
 4   mileage       10781 non-null  int64  
 5   fuelType      10781 non-null  object 
 6   tax           10781 non-null  int64  
 7   mpg           10781 non-null  float64
 8   engineSize    10781 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 758.2+ KB


In [4]:
# Duplicates checking and removal
print("missing values:")
print(df.isna().sum())

df = df.drop_duplicates()
print("after drop_duplicates:", df.shape)



missing values:
model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64
after drop_duplicates: (10664, 9)


In [5]:
# Feature Engineering that might help with the prediction (car age, mileage per year)

REFERENCE_YEAR = 2025  
df_fe = df.copy()

df_fe["age"] = (REFERENCE_YEAR - df_fe["year"]).clip(lower=0)
df_fe["mileage_per_year"] = df_fe["mileage"] / df_fe["age"].replace(0, 1)

feature_cols_fe = [
    "model",
    "transmission",
    "mileage",
    "fuelType",
    "tax",
    "mpg",
    "engineSize",
    "age",
    "mileage_per_year",
]

X_fe = df_fe[feature_cols_fe]
y = df_fe["price"]

X_fe.head()


,model,transmission,mileage,fuelType,tax,mpg,engineSize,age,mileage_per_year
0,5 Series,Automatic,67068,Diesel,125,57.6,2.0,11,6097.090909
1,6 Series,Automatic,14827,Petrol,145,42.8,2.0,7,2118.142857
2,5 Series,Automatic,62794,Diesel,160,51.4,3.0,9,6977.111111
3,1 Series,Automatic,26676,Diesel,145,72.4,1.5,8,3334.500000
4,7 Series,Automatic,39554,Diesel,160,50.4,3.0,11,3595.818182


In [6]:
# Feature and target seperation (x=features, y=target (price))

target = "price"
feature_cols = ["model", "year", "transmission", "mileage", "fuelType", "tax", "mpg", "engineSize"]

X = df[feature_cols]
y = df[target]

X.head()


,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,Automatic,39554,Diesel,160,50.4,3.0


In [7]:
# Feature inspection, each feature and its unique values count

for col in ["model", "transmission", "fuelType"]:
    print(col, "unique values:", X[col].nunique())
    print(X[col].value_counts().head(), "\n")


model unique values: 24
model
3 Series    2434
1 Series    1962
2 Series    1188
5 Series    1051
4 Series     986
Name: count, dtype: int64 

transmission unique values: 3
transmission
Semi-Auto    4637
Automatic    3542
Manual       2485
Name: count, dtype: int64 

fuelType unique values: 5
fuelType
Diesel      6988
Petrol      3340
Hybrid       297
Other         36
Electric       3
Name: count, dtype: int64 



In [8]:
# Train Test Split (80% for training, 20% for evaluation)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_fe, y, test_size=0.2, random_state=42
)

print(X_train.shape, X_test.shape)


(8531, 9) (2133, 9)


In [9]:
#Preprocessing Pipeline(Seperates categorical and numerical features)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_features = ["model", "transmission", "fuelType"]
numeric_features = ["mileage", "tax", "mpg", "engineSize", "age", "mileage_per_year"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numeric_features),
    ]
)

preprocessor


,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('cat', ...), ('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [10]:
# Model Evaluation Function(uses: MAE, EMSE, R^2 score)
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)
    return mae, rmse, r2


In [11]:
# Advanced Model Pipeline and Hyperparameter Tuning

import numpy as np
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.pipeline import Pipeline

cv = KFold(n_splits=5, shuffle=True, random_state=42)

base_model = ExtraTreesRegressor(random_state=42, n_jobs=-1)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", base_model),
    ]
)

tt_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1,
)

param_grid = {
    "regressor__model__n_estimators": [300, 600, 900],
    "regressor__model__max_depth": [None, 20, 40, 60],
    "regressor__model__min_samples_split": [2, 5, 10],
    "regressor__model__min_samples_leaf": [1, 2, 4],
    "regressor__model__max_features": ["sqrt", "log2", 0.7],
}

search = RandomizedSearchCV(
    tt_model,
    param_distributions=param_grid,
    n_iter=20,
    scoring="neg_mean_absolute_error",
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1,
)


In [12]:
# Fit the search
search.fit(X_train, y_train)

print("Best MAE (CV):", -search.best_score_)
print("Best params:", search.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best MAE (CV): 1563.4580373990952
Best params: {'regressor__model__n_estimators': 900, 'regressor__model__min_samples_split': 5, 'regressor__model__min_samples_leaf': 1, 'regressor__model__max_features': 0.7, 'regressor__model__max_depth': 60}


In [13]:
# Evaluate tuned model on the held-out test set
best_tuned = search.best_estimator_

preds = best_tuned.predict(X_test)
mae = mean_absolute_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print("Tuned model -> MAE:", mae, "RMSE:", rmse, "R2:", r2)


Tuned model -> MAE: 1515.398467389196 RMSE: 2642.6033799003344 R2: 0.9439549929460442


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Force dense output for HGB (Histogram Gradient Boosting)
try:
    ohe_dense = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe_dense = OneHotEncoder(handle_unknown="ignore", sparse=False)

preprocessor_dense = ColumnTransformer(
    transformers=[
        ("cat", ohe_dense, categorical_features),
        ("num", "passthrough", numeric_features),
    ],
    sparse_threshold=0.0,
)


In [16]:
#  Histogram Gradient Boosting Model Training and Optimization
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline

cv = KFold(n_splits=5, shuffle=True, random_state=42)

hgb = HistGradientBoostingRegressor(random_state=42)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor_dense),
        ("model", hgb),
    ]
)

tt_model = TransformedTargetRegressor(
    regressor=pipeline,
    func=np.log1p,
    inverse_func=np.expm1,
)

param_grid = {
    "regressor__model__max_iter": [200, 400, 800],
    "regressor__model__learning_rate": [0.03, 0.05, 0.1],
    "regressor__model__max_depth": [None, 6, 10],
    "regressor__model__max_leaf_nodes": [31, 63, 127],
    "regressor__model__min_samples_leaf": [20, 50, 100],
    "regressor__model__l2_regularization": [0.0, 0.1, 1.0],
}

search_hgb = RandomizedSearchCV(
    tt_model,
    param_distributions=param_grid,
    n_iter=25,
    scoring="neg_mean_absolute_error",
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1,
)

search_hgb.fit(X_train, y_train)

print("Best MAE (CV):", -search_hgb.best_score_)
print("Best params:", search_hgb.best_params_)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best MAE (CV): 1524.7468662093556
Best params: {'regressor__model__min_samples_leaf': 20, 'regressor__model__max_leaf_nodes': 31, 'regressor__model__max_iter': 800, 'regressor__model__max_depth': None, 'regressor__model__learning_rate': 0.05, 'regressor__model__l2_regularization': 0.1}


In [17]:
# Final Model Evaluation on Test Data

best_hgb = search_hgb.best_estimator_

preds = best_hgb.predict(X_test)
mae = mean_absolute_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print("HGB tuned -> MAE:", mae, "RMSE:", rmse, "R2:", r2)


HGB tuned -> MAE: 1429.4768699783447 RMSE: 2485.3182839155243 R2: 0.9504279378397541


In [18]:
# Adds the features to the the dataframe

def add_features(df, reference_year=REFERENCE_YEAR):
    df = df.copy()
    df["age"] = (reference_year - df["year"]).clip(lower=0)
    df["mileage_per_year"] = df["mileage"] / df["age"].replace(0, 1)
    return df


In [19]:
# Predicts prices for new bmw cars 

best_model = best_hgb  # use the tuned HGB model

sample = pd.DataFrame([
    {
        "model": "5 Series",
        "year": 2018,
        "transmission": "Automatic",
        "mileage": 23195,
        "fuelType": "Diesel",
        "tax": 145,
        "mpg": 65.7,
        "engineSize": 2.0,
    },
    {
        "model": "1 Series",
        "year": 2017,
        "transmission": "Manual",
        "mileage": 17117,
        "fuelType": "Petrol",
        "tax": 145,
        "mpg": 53.3,
        "engineSize": 1.5,
    },
    {
        "model": "X3",
        "year": 2016,
        "transmission": "Automatic",
        "mileage": 42691,
        "fuelType": "Diesel",
        "tax": 145,
        "mpg": 54.3,
        "engineSize": 2.0,
    },
])

sample_fe = add_features(sample)
preds = best_model.predict(sample_fe[feature_cols_fe])

sample_out = sample.copy()
sample_out["predicted_price"] = preds.round(0).astype(int)
sample_out


,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,predicted_price
0,5 Series,2018,Automatic,23195,Diesel,145,65.7,2.0,20443
1,1 Series,2017,Manual,17117,Petrol,145,53.3,1.5,16671
2,X3,2016,Automatic,42691,Diesel,145,54.3,2.0,17993
